In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import geopandas as gpd
import descartes
import matplotlib.pyplot as plt
%matplotlib inline
import sys
sys.path.insert(0, '../src')
import fetch
import clean

In [ ]:
stop_2014 = fetch.fetch_old(2014)
stop_2015 = fetch.fetch_old(2015)
stop_2016 = fetch.fetch_old(2016)
stop_2017 = fetch.fetch_old(2017)
stop_2018 = fetch.fetch_new(2018)
stop_2019 = fetch.fetch_new(2019)

In [ ]:
# standardize the columns so that won't get too many NaN cols
COLUMNS = ["stop_id","pid","beat","service_area","stop_cause"\
    ,"subject_race","subject_age","subject_sex","arrested",\
    "searched","property_seized","contraband_found","date_stop","time_stop","reason_for_stop_explanation"]

stop_2014 = clean.standardize(stop_2014,COLUMNS)
stop_2015 = clean.standardize(stop_2015,COLUMNS)
stop_2016 = clean.standardize(stop_2016,COLUMNS)
stop_2017 = clean.standardize(stop_2017,COLUMNS)
stop_2018 = clean.standardize(stop_2018,COLUMNS)
stop_2019 = clean.standardize(stop_2019,COLUMNS)

# clean the cols
for c in clean.old_clean.keys():
    stop_2014[c] = stop_2014[c].apply(clean.old_clean[c])
    stop_2015[c] = stop_2015[c].apply(clean.old_clean[c])
    stop_2016[c] = stop_2016[c].apply(clean.old_clean[c])
    stop_2017[c] = stop_2017[c].apply(clean.old_clean[c])
for c in clean.new_clean.keys():
    stop_2018[c] = stop_2018[c].apply(clean.new_clean[c])
    stop_2019[c] = stop_2019[c].apply(clean.new_clean[c])

# we first separate the data by before RML and after RML (Nov. 2016)
before_2016 = stop_2016[pd.to_datetime(stop_2016.date_stop).dt.month < 11]
after_2016 = stop_2016[pd.to_datetime(stop_2016.date_stop).dt.month > 11]


In [ ]:
before = pd.concat([stop_2014,stop_2015,before_2016])
after = pd.concat([after_2016,stop_2017,stop_2018,stop_2019])
total = pd.concat([stop_2014,stop_2015,stop_2016,stop_2017,stop_2018,stop_2019])

In [ ]:
before_dui = clean.filter_to_dui(before)
after_dui = clean.filter_to_dui(after)
total_dui = clean.filter_to_dui(total)

##### the DUI percentage among stop data is increased after RML

In [ ]:
len(before_dui)/len(before)

In [ ]:
len(after_dui)/len(after)

##### looking at yearly/monthly trend

In [ ]:
before_dui["YEAR"] = before_dui.date_stop.apply(lambda x:int(x[:4]))
after_dui["YEAR"] = after_dui.date_stop.apply(lambda x:int(x[:4]))
before_dui['MONTH'] = before_dui.date_stop.apply(lambda x:int(x.split("-")[1]))
after_dui['MONTH'] = after_dui.date_stop.apply(lambda x:int(x.split("-")[1]))
total_dui["YEAR"] = total_dui.date_stop.apply(lambda x:int(x[:4]))
total_dui['MONTH'] = total_dui.date_stop.apply(lambda x:int(x.split("-")[1]))

In [ ]:
total_dui.groupby("YEAR").size().plot()

In [ ]:
total_dui.groupby(["YEAR",'MONTH']).size().plot()

In [ ]:
stop_1617 = pd.concat([stop_2016,stop_2017])

In [ ]:
stop_1617['YEAR'] = stop_1617.date_stop.apply(lambda x:int(x[:4]))
stop_1617['MONTH'] = stop_1617.date_stop.apply(lambda x:int(x.split("-")[1]))

In [ ]:
(clean.filter_to_dui(stop_1617).groupby(['YEAR','MONTH']).size()/stop_1617.groupby(['YEAR','MONTH']).size()).plot()

In [ ]:
collision = fetch.fetch_collision()

In [ ]:
collision_dui = clean.collision_dui(collision)

In [ ]:
collision_dui['YEAR'] = collision_dui.date_time.apply(lambda x:int(x[:4]))
collision_dui['MONTH'] = collision_dui.date_time.apply(lambda x:int(x.split("-")[1]))


collision['YEAR'] = collision.date_time.apply(lambda x:int(x[:4]))
collision['MONTH'] = collision.date_time.apply(lambda x:int(x.split("-")[1]))


In [ ]:
collision_dui.groupby(["YEAR","MONTH"]).size().plot()

In [ ]:
(collision_dui.groupby(["YEAR","MONTH"]).size()/collision.groupby(["YEAR","MONTH"]).size()).plot()

#### collision data (SWITRS)

In [ ]:
df = pd.read_csv("../data/collision/CollisionRecords.txt")
df = df[~df.LATITUDE.isna()]
df.LONGITUDE = -df.LONGITUDE
df = df[(df.LONGITUDE > -117.25) & (df.LONGITUDE < -116.8)]
df = df[df.LATITUDE <= 33.2]

In [ ]:
gdf = gpd.GeoDataFrame(df,geometry = gpd.points_from_xy(df.LONGITUDE,df.LATITUDE))

In [ ]:
sd = gpd.read_file("../../DSC180/data/police_beat/pd_beats_datasd.shp")

In [ ]:
gdf.crs = 'epsg:4326'
sd = sd.to_crs({'init': 'epsg:4326'})

In [ ]:
base = sd.plot()
gdf.plot(ax=base,color='black', markersize=3)

PA collision data (deprecatedd)

In [ ]:
a = pd.read_csv("../data/PA_Collision/FLAG_2015_Statewide.csv")
a = a[['CRN','ALCOHOL_RELATED','DRUG_RELATED']]
a['DUI'] = a.apply(lambda x:np.any([x.ALCOHOL_RELATED,x.DRUG_RELATED]),axis=1)

In [ ]:
b = pd.read_csv("../data/PA_Collision/FLAG_2016_Statewide.csv")
b = b[['CRN','ALCOHOL_RELATED','DRUG_RELATED']]
b['DUI'] = b.apply(lambda x:np.any([x.ALCOHOL_RELATED,x.DRUG_RELATED]),axis=1)

In [ ]:
c = pd.read_csv("../data/PA_Collision/FLAG_2019.csv")
c = c[['CRN','ALCOHOL_RE','DRUG_RELAT']]
c = c.replace({'No':0,'Yes':1})
c['DUI'] = c.apply(lambda x:np.any([x.ALCOHOL_RE,x.DRUG_RELAT]),axis=1)

In [ ]:
d = pd.read_csv("../data/PA_Collision/FLAG_V_2018.csv")
d = d[['CRN','ALCOHOL_RELATED','DRUG_RELATED']]
d = d.replace({'No':0,'Yes':1})
d['DUI'] = d.apply(lambda x:np.any([x.ALCOHOL_RELATED,x.DRUG_RELATED]),axis=1)

In [ ]:
a_dui = a.DUI.sum()/len(a)
b_dui = b.DUI.sum()/len(b)
c.DUI.sum()/len(c)
d.DUI.sum()/len(d)


In [ ]:
plt.plot([2015,2016,2017,2018],[a.DUI.sum()/len(a),b.DUI.sum()/len(b),c.DUI.sum()/len(c),d.DUI.sum()/len(d)])

In [ ]:
ca = pd.read_csv("../data/CA_Collision/CollisionRecords.txt",error_bad_lines=False)

In [ ]:
ca = ca[['CASE_ID','ACCIDENT_YEAR','COLLISION_DATE','PCF_VIOL_CATEGORY','ALCOHOL_INVOLVED']]
ca = ca[(ca.ACCIDENT_YEAR > 2014)&(ca.ACCIDENT_YEAR < 2019)]

In [ ]:
ca_2015 = ca[ca.ACCIDENT_YEAR == 2015]
ca_2016 = ca[ca.ACCIDENT_YEAR == 2016]
ca_2017 = ca[ca.ACCIDENT_YEAR == 2017]
ca_2018 = ca[ca.ACCIDENT_YEAR == 2018]


In [ ]:
ca_2015_dui = ca_2015[ca_2015.PCF_VIOL_CATEGORY == "01"].size/len(ca_2015)
ca_2016_dui = ca_2016[ca_2016.PCF_VIOL_CATEGORY == "01"].size/len(ca_2016)
ca_2017_dui = ca_2017[ca_2017.PCF_VIOL_CATEGORY == "01"].size/len(ca_2017)
ca_2018_dui = ca_2018[ca_2018.PCF_VIOL_CATEGORY == "01"].size/len(ca_2018)

In [ ]:
plt.plot([2015,2016,2017,2018],[ca_2015_dui,ca_2016_dui,ca_2017_dui,ca_2018_dui])

In [ ]:
ca["MONTH"] = ca.COLLISION_DATE.apply(lambda x:int(str(x)[4:6]))

In [ ]:
(ca[ca.PCF_VIOL_CATEGORY == "01"].groupby(['ACCIDENT_YEAR','MONTH']).size()/ca.groupby(['ACCIDENT_YEAR','MONTH']).size()).plot()



In [ ]:
sd_2015_p = pd.read_csv("../data/SD_Collision/Person2015.txt",sep="|")
sd_2015_a = pd.read_csv("../data/SD_Collision/Accident2015.txt",sep="|")
sd_2015_a = sd_2015_a[['AccidentSeqID','AccidentDateTime']]
sd_2015_p = sd_2015_p[["AccidentSeqID","DrugUseDesc",'AlcoholUseDesc']]
sd_2015 = pd.merge(left=sd_2015_a,right=sd_2015_p,on='AccidentSeqID')

In [ ]:
sd_2016_p = pd.read_csv("../data/SD_Collision/Person2016.txt",sep="|")
sd_2016_a = pd.read_csv("../data/SD_Collision/Accident2016.txt",sep="|")
sd_2016_a = sd_2016_a[['AccidentSeqID','AccidentDateTime']]
sd_2016_p = sd_2016_p[["AccidentSeqID","DrugUseDesc",'AlcoholUseDesc']]
sd_2016 = pd.merge(left=sd_2016_a,right=sd_2016_p,on='AccidentSeqID')

In [ ]:
sd_2017_p = pd.read_csv("../data/SD_Collision/Person2017.txt",sep="|")
sd_2017_a = pd.read_csv("../data/SD_Collision/Accident2017.txt",sep="|")
sd_2017_a = sd_2017_a[['AccidentSeqID','AccidentDateTime']]
sd_2017_p = sd_2017_p[["AccidentSeqID","DrugUseDesc",'AlcoholUseDesc']]
sd_2017 = pd.merge(left=sd_2017_a,right=sd_2017_p,on='AccidentSeqID')

In [ ]:
sd_2018_p = pd.read_csv("../data/SD_Collision/Person2018.txt",sep="|")
sd_2018_a = pd.read_csv("../data/SD_Collision/Accident2018.txt",sep="|")
sd_2018_a = sd_2018_a[['AccidentSeqID','AccidentDateTime']]
sd_2018_p = sd_2018_p[["AccidentSeqID","DrugUseDesc",'AlcoholUseDesc']]
sd_2018 = pd.merge(left=sd_2018_a,right=sd_2018_p,on='AccidentSeqID')

In [ ]:
sd_2015['DRUG'] = sd_2015.DrugUseDesc.apply(lambda x:1 if x == 'Drugs used' else 0)
sd_2015['ALCOHOL'] = sd_2015.AlcoholUseDesc.apply(lambda x:1 if x == 'Alcohol used' else 0)
sd_2015['DUI'] = sd_2015.apply(lambda x:np.any([x.DRUG,x.ALCOHOL]),axis=1)
sd_2015['MONTH'] = pd.to_datetime(sd_2015.AccidentDateTime).dt.month
sd_2015['YEAR'] = 2015

sd_2016['DRUG'] = sd_2016.DrugUseDesc.apply(lambda x:1 if x == 'Drugs used' else 0)
sd_2016['ALCOHOL'] = sd_2016.AlcoholUseDesc.apply(lambda x:1 if x == 'Alcohol used' else 0)
sd_2016['DUI'] = sd_2016.apply(lambda x:np.any([x.DRUG,x.ALCOHOL]),axis=1)
sd_2016['MONTH'] = pd.to_datetime(sd_2016.AccidentDateTime).dt.month
sd_2016['YEAR'] = 2016


sd_2017['DRUG'] = sd_2017.DrugUseDesc.apply(lambda x:1 if x == 'Drugs used' else 0)
sd_2017['ALCOHOL'] = sd_2017.AlcoholUseDesc.apply(lambda x:1 if x == 'Alcohol used' else 0)
sd_2017['DUI'] = sd_2017.apply(lambda x:np.any([x.DRUG,x.ALCOHOL]),axis=1)
sd_2017['MONTH'] = pd.to_datetime(sd_2017.AccidentDateTime).dt.month
sd_2017['YEAR'] = 2017


sd_2018['DRUG'] = sd_2018.DrugUseDesc.apply(lambda x:1 if x == 'Drugs used' else 0)
sd_2018['ALCOHOL'] = sd_2018.AlcoholUseDesc.apply(lambda x:1 if x == 'Alcohol used' else 0)
sd_2018['DUI'] = sd_2018.apply(lambda x:np.any([x.DRUG,x.ALCOHOL]),axis=1)
sd_2018['MONTH'] = pd.to_datetime(sd_2018.AccidentDateTime).dt.month
sd_2018['YEAR'] = 2018


In [ ]:
sd_2015_dui = sd_2015.DUI.sum()/len(sd_2015)
sd_2016_dui = sd_2016.DUI.sum()/len(sd_2016)
sd_2017_dui = sd_2017.DUI.sum()/len(sd_2017)
sd_2018_dui = sd_2018.DUI.sum()/len(sd_2018)

In [ ]:
plt.plot([2015,2016,2017,2018],[sd_2015_dui,sd_2016_dui,sd_2017_dui,sd_2018_dui])

In [ ]:
sd = pd.concat([sd_2015,sd_2016,sd_2017,sd_2018])

In [ ]:
(sd[sd.DUI == 1].groupby(['YEAR',"MONTH"]).size()/sd.groupby(['YEAR',"MONTH"]).size()).plot()